https://stepik.org/lesson/309343/step/6


входной тензор править не нужно, он обрабатывается за кадром. В задании требуется из ядра W построить тензор W', т.е. расположить красные квадратики среди белых нулей. Я рекомендую сначала вычислить выходные размеры, создать выходной тензор из нулей и в нем нужные позиции замещать значениями из исходного ядра. Но, разумеется, можно и по-другому.

[Mastering Tensor Padding in PyTorch: A Guide to Reflect and Replicate](https://medium.com/aimonks/mastering-tensor-padding-in-pytorch-a-guide-to-reflect-and-replicate-441b4fa8b0b4)

In [1]:
#@title Решение
import torch
import torch.nn.functional as F
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=1,
                      input_height=4, input_width=4, stride=2): # размеры задаём здесь для удобства
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1] # из ядра получаем число каналов для генерации батча с картинками
    # генерируем батч
    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width) # создаём ему нужную форму
    # глянем размеры батча
    # print('input_tensor.shape (картинок в батче, каналов, размеры картинки) = ', input_tensor.shape)
    # print('input_tensor = ', input_tensor)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    # Approximate equality comparison
    # https://www.slingacademy.com/article/pytorch-how-to-compare-2-tensors/
    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        X = torch_input
        W = self.kernel
        padding = 0

        import numpy as np
        # посчитаем высоту и ширину развёрнутого ядра
        stride = self.stride
        out_height = ((X.shape[2] - W.shape[2] + 2 * padding) // stride + 1) ** 2 # высота (разворот обычного выхода)
        out_width = X.shape[2] ** 2 * X.shape[1]                           # ширина (разм. картинки на число каналов)
        # print('out_height, out_width =', out_height, out_width)

        # задаём шаблон развёрнутой матрицы (пока из нулей)
        W_prime = np.zeros((out_height, out_width))
        # print('W_prime_zeros.shape =', W_prime.shape)
        # print('W_prime_zeros =\n', W_prime)

        # дополним матрицу исходного ядра нулями справа, если необходимо
        # https://medium.com/aimonks/mastering-tensor-padding-in-pytorch-a-guide-to-reflect-and-replicate-441b4fa8b0b4
        padding = (0, 1, 0, 1, 0, 0, 0, 0)
        W_pad = F.pad(W, padding, mode='constant', value=0)
        # print('W_pad.shape =', W_pad.shape)
        # print('W_pad =\n', W_pad)

        # разворачиваем исходное ядро в одномерный массив
        W_line = W_pad.reshape(-1)
        # print('W_line.shape =', W_line.shape)

        # обрезаем последний ноль, чтобы не выйти за пределы индекса в последней строке
        W_line = W_line[:-1]
        # print('W_line.shape =', W_line.shape)
        # print('W_line =\n', W_line)

        indent = 0 # начальный отступ
        # проход по строкам развёрнутой заготовки
        for i in range(W_prime.shape[0]):
            # проход по элементам строки
            for j in range(W_line.shape[0]):
                W_prime[i, indent + j] = W_line[j]
            # увеличиваем отступ слева
            if (indent + W.shape[0]) % X.shape[0] == 0:
                indent += X.shape[0] - 1
            else:
                indent += 2
        # print('W_prime =\n', W_prime)
        return torch.tensor(W_prime).float()


    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrix))

True


In [2]:
#@title кусочек кода, генерирующий входной тензор:
# conv2d_layer_class, batch_size=2, input_height=4, input_width=4, stride=2
import torch

# batch_size=2
# in_channels = kernel.shape[1]
# # in_channels=3
# input_height=4
# input_width=4
# stride=2

# input_tensor = torch.arange(0, batch_size * in_channels *
#                                 input_height * input_width,
#                                 out=torch.FloatTensor()) \
#         .reshape(batch_size, in_channels, input_height, input_width)

# input_tensor

In [3]:
#@title кусочек кода, задающий ядро:
import torch

kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

kernel

tensor([[[[ 0.,  1.,  0.],
          [ 1.,  2.,  1.],
          [ 0.,  1.,  0.]],

         [[ 1.,  2.,  1.],
          [ 0.,  3.,  3.],
          [ 0.,  1., 10.]],

         [[10., 11., 12.],
          [13., 14., 15.],
          [16., 17., 18.]]]])

In [4]:
#@title Ядро свёртки с размерами (out_channels, in_channels, kernel_height, kernel_width)
kernel = torch.tensor(
    [[[[0., 1, 0],
       [1,  2, 1],
       [0,  1, 0]],

      [[1, 2, 1],
       [0, 3, 3],
       [0, 1, 10]],

      [[10, 11, 12],
       [13, 14, 15],
       [16, 17, 18]]]])

kernel_padding = torch.nn.functional.pad(kernel, (0, 2*2-kernel.shape[3], 0, 0)).view(1, -1)
kernel_padding = torch.nn.functional.pad(kernel_padding, (0, 12, 0, 0))
print(kernel_padding.shape)
kernel_padding

# for i in range(kernel.shape[0]):
#     for j in range(kernel.shape[1]):
#         for k in range(kernel.shape[2]):
#             print(kernel[i, j, k])


#


torch.Size([1, 48])


tensor([[ 0.,  1.,  0.,  0.,  1.,  2.,  1.,  0.,  0.,  1.,  0.,  0.,  1.,  2.,
          1.,  0.,  0.,  3.,  3.,  0.,  0.,  1., 10.,  0., 10., 11., 12.,  0.,
         13., 14., 15.,  0., 16., 17., 18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.]])

In [12]:
#@title Функция преобразование кернела в матрицу нужного вида batch_size=1,
batch_size, out_channels, output_height, output_width = 1, 1, 3, 3
in_channels = kernel.shape[1]
input_height = 4
input_width = 4
input_tensor = torch.arange(0, batch_size * in_channels *
                            input_height * input_width,
                            out=torch.FloatTensor()) \
    .reshape(batch_size, in_channels, input_height, input_width) # создаём ему нужную форму

# Функция преобразование кернела в матрицу нужного вида.
def unsqueeze_kernel(self, torch_input, output_height, output_width):
        # Реализуйте функцию, возвращающую преобразованный кернел
        kernel_unsqueezed_zeros = torch.zeros((batch_size, input_tensor.shape[1] * input_tensor.shape[2] * out_channels))
        input_height = torch_input.shape[2]
        input_width = torch_input.shape[3]
        kernel_unsqueezed = torch.zeros((input_height, input_width, output_height, output_width))
        # TODO
        return kernel_unsqueezed

kernel_unsqueezed = unsqueeze_kernel(kernel, input_tensor, output_height, output_width)
kernel_unsqueezed
#

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
        